In [68]:
# parse SRA metadata
import pandas as pd
data = pd.read_csv('SraRunTable.txt', header = 0, sep = '\t')
sample_id = data['Sample_Name'].tolist()
sra_id = data['Run'].tolist()

label = pd.read_csv('Gevers_meta_data.csv', header = 0, sep = ',')
sample_id_label = label['sample'].tolist()
pcdai_label = label['PCDAI'].tolist()
diag_label = label['Diagnosis'].tolist()

sample_id_label = [item.replace('-', '').replace('.', '') for item in sample_id_label]
sample2sra = {}
for sample_id_item, sra_id_item in zip(sample_id, sra_id):
    sample_id_item = sample_id_item.replace('_', '').replace('-', '')
    if '.' in sample_id_item:
        sample_id_item = sample_id_item.split('.')[1]
    if sample_id_item in sample_id_label:
        sample2sra[sample_id_item] = sra_id_item
        
sra_list = []
diag_list = []
pcdai_list = []
for sample_id_label_item, diag_label_item, pcdai_label_item in zip(sample_id_label, diag_label, pcdai_label):
    sra_list.append(sample2sra[sample_id_label_item])
    diag_list.append(diag_label_item)
    pcdai_list.append(pcdai_label_item)
d = {'SRA': sra_list, 'Diag': diag_list, 'PCDAI': pcdai_list}
df = pd.DataFrame(data=d)    
df.to_csv('meta_data_cleaned.csv')

In [13]:
# download
import pandas as pd
import ftplib

meta_data= pd.read_csv('meta_data_cleaned.csv', header = 0, sep = ',')
sra_list = meta_data['SRA'].tolist()
sever_ip = 'ftp-trace.ncbi.nlm.nih.gov'

def ftp_download(sever_ip, path, filename):
    
    ftp = ftplib.FTP(sever_ip) 
    ftp.login() 
    ftp.cwd(path)
    ftp.retrbinary("RETR " + filename, open('sra_data/' + filename, 'wb').write)
    ftp.quit()
    
    return


fw = open('LOG.DOWNLOAD_SRA.txt', 'w+')
for sra_id in sra_list[:2]:
    path = 'sra/sra-instant/reads/ByStudy/sra/SRP/SRP040/SRP040765/' + sra_id + '/'
    filename = sra_id + '.sra'
    try:
        ftp_download(sever_ip, path, filename)
    except:
        fw.write(filename + ' failed' + '\n')
fw.close() 